In [ ]:
str_list = ['aaa','bbb']
t1 = 'aaa'
t1 in str_list

In [ ]:
from glob import glob
from tqdm import tqdm
import os
import numpy as np
import xml.etree.ElementTree as ET
from random import choices
from utils import crossover, find_if_in_list, find_unique_xmls, mutation, find_size_by_name
xml_files = glob('./ori_population/*.xml')
xml_list = []
for xml_file in tqdm(xml_files):
    tree = ET.parse(xml_file)
    xml_list.append(tree)

In [ ]:
fitness_list = []
for tree in tqdm(xml_list):
    tree.write('./temp.xml')
    output = os.popen(f'./bin/burrTxt -d ./temp.xml').read()
    output = output.split('-------------------------------------------------------')
    levels = output[-1].split('\n')[1:-2]
    levels = [x.replace('level: ','') for x in levels]
    levels = [sum([int(y) for y in x.split('.')]) for x in levels]
    fitness = np.mean(levels)
    fitness_list.append(fitness)

In [ ]:
import matplotlib.pyplot as plt
fitness_list = np.array(fitness_list)
fitness_list = np.nan_to_num(fitness_list, nan=0)
plt.plot(fitness_list)

In [ ]:
temprature = 0.8
exp_list = np.exp(fitness_list/temprature)
prob = exp_list / np.sum(exp_list)
plt.plot(prob)

In [ ]:
prob = fitness_list/np.sum(fitness_list)
for _ in range(1000):
    parent_a = choices(xml_list, weights=prob, k=1)
    parent_b = choices(xml_list, weights=prob, k=1)
    xml = crossover(parent_a[0], parent_b[0], 'templates/3P_5x5')
    xml.write('./temp.xml')
    output = os.popen('./bin/burrTxt -d -q ./temp.xml').read()
    # if 'units too many for the result shape' not in output:
    # # output = output.split(' ')
    #     print(output)

In [ ]:
# fitness_mean = np.mean(fitness_list)
# fitness_std = np.std(fitness_list)
# fitness_list = (fitness_list - fitness_mean) / (fitness_std + 0.00001)
temprature = 0.8
exp_list = np.exp(fitness_list/temprature)
prob = exp_list / np.sum(exp_list)
# prob = fitness_list/np.sum(fitness_list)
offsrping_number = 100
offspring_size = 0
offspring_list = []
while True:
    parent_a = choices(xml_list, weights=prob, k=1)
    parent_b = choices(xml_list, weights=prob, k=1)
    xml = crossover(parent_a[0], parent_b[0], 'templates/4P_5x5')
    xml.write('./temp.xml')
    output = os.popen('./bin/burrTxt -d -q ./temp.xml').read()
    output = output.split(' ')
    # print(output)
    if output[3] != '0' and output[3] != 'be' and output[3] != 'many' and output[3] != 'few':
        offspring_list.append(xml)
        offspring_size += 1
        if offspring_size > offsrping_number:
            break
        print(offspring_size / offsrping_number * 100)
for i in tqdm(range(len(offspring_list))):
    if np.random.uniform(0, 1) < 0.2:
        # 20% mutation
        while True:
            xml = mutation(offspring_list[i], 'templates/4P_5x5', True)
            xml.write('./temp.xml')
            output = os.popen('./bin/burrTxt -d -q ./temp.xml').read()
            output = output.split(' ')
            if output[3] != '0' and output[3] != 'be' and output[3] != 'many' and output[3] != 'few':
                print(output)
                offspring_list[i] = xml
                break
fitness_list = []
xml_list = offspring_list
xml_list = find_unique_xmls(xml_list)
for tree in tqdm(xml_list):
        tree.write('./temp.xml')
        output = os.popen(f'./bin/burrTxt -d ./temp.xml').read()
        output = output.split('-------------------------------------------------------')
        levels = output[-1].split('\n')[1:-2]
        levels = [x.replace('level: ','') for x in levels]
        levels = [sum([int(y) for y in x.split('.')]) for x in levels]
        # fitness = max(levels)
        fitness = np.mean(levels)
        fitness_list.append(fitness)
fitness_list = np.array(fitness_list)
fitness_list = np.nan_to_num(fitness_list, nan=0)

In [ ]:
plt.plot(fitness_list)